<a href="https://colab.research.google.com/github/FredArgoX/ChaoticTest_Keras/blob/main/Ex_01_Image_classification_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source:
[Image classification from scratch
](https://keras.io/examples/vision/image_classification_from_scratch/)

# Image classification from scratch

Description: Training an image classifier from scratch on the Kaggle Cats vs Dogs dataset.

# Introduction

This example shows how to do image classification fro scratch, starting from JPEG image files on disk, without leveraging pre-trained weights or a pre-made Keras Application model. We demonstrate the workflow on the Kaggle Cats vs Dogs binary classification dataset.

We use the image_dataset_from_directory utility to generate the datasets, and we use Keras image preprocessing layers for image standarization and data augmentation.

# Setup

In [1]:
import os
import numpy as np
import keras
from keras import layers
from tensorflow import data as tf_data
import matplotlib.pyplot as plt

# Load the data: the Cats vs Dogs dataset

# Generate a dataset

# Visualize the data

# Using image data augmentation

# Standarizing the data

# Configure the dataset for performance

# Build a model

# Train the model

# Run inference on new data